<a href="https://colab.research.google.com/github/MatheusOrange211/Bootcamp-Alura/blob/main/PROJETO_MODULO_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PERGUNTA :
COMO ESTÁ A SAÚDE PSICOLÓGICA DOS ALUNOS?

> A adolescência é um período muito importante na vida das pessoas. Neste período, enfrentamos os primeiros desafios da vida adulta, ainda que em grau menor. Passamos por altos e baixos na autoestima e nas relações ao nosso redor. O projeto abaixo tem como objetivo, de forma simples, apenas tentar entender melhor o comportamento dos alunos que foram entrevistados e buscar entender melhor a forma como agem com amigos e familiares e como estes encontram-se em suas vidas. Buscar entender, ainda que superficialmente como são as relações ao seu redor em cada idade.



#IMPORTANDO BIBLIOTECAS E CONFIGURAÇÕES INICIAIS

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#IMPORTANDO OS DADOS

In [2]:
arquivos = "/content/drive/MyDrive/bootcamp-modulo3/PeNSE_2015_AMOSTRA2.zip"
zip = ZipFile(arquivos)
amostra_alunoescola = zip.namelist()[1]

In [3]:
dados = pd.read_csv(zip.open(amostra_alunoescola),sep=';',thousands='.',decimal=',')
dados

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,VB01007,VB01008A,VB01010A,VB01011,VB01012,VB01013,VB01014,VB01015A,VB01016,VB01017,VB01018,VB01019,VB01020A,VB01021,VB01022,VB01023,VB01024,VB01025,VB01026,VB02001,VB02002,VB02004A,VB02010,VB02011,VB02013,VB02017A,VB02018A,VB02019A,VB02020A,VB02021,...,VB11005,VB11006,VB11007,VB12001,VB12002,VB12003,VB13001,VB13002A,VB13004A,VB13005,VB13006,VB13007,VB13008,VB13009,VB14001,VB14002,VB16001A01,VB16001A02,VB16001A03,VB16001A04,VB16001A05,VB16001A06,VB16001A07,VB16001A08,VB17001,VB17002,VB17003,VB17004,VB17005,VB17006,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,2,7,3,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,5,1,8,3,4,3,6,6,1,6,1,-1,2,...,2,1,2,1,1,4,2,-1,3,1,1,-1,1,1,2,2,1,2,2,2,2,2,2,2,371.0,1570.0,37.1,157.0,2,1,1223,1,299.735235,1,2,4,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,1,7,3,2,-1,1,1,1,1,1,2,2,2,4,1,2,2,5,3,8,2,1,8,8,6,1,1,5,-1,2,...,2,3,2,4,4,4,1,2,3,3,1,-1,1,1,2,2,2,2,2,1,2,2,2,2,502.0,1520.0,50.2,152.0,2,2,1223,1,355.170081,1,2,4,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,1,5,5,2,-1,2,1,1,1,1,2,3,1,4,1,2,2,5,3,2,3,1,8,2,8,1,1,5,5,1,...,2,1,2,1,2,3,1,2,1,2,5,99,1,-1,2,2,1,1,1,1,1,1,1,1,482.0,1614.0,48.2,161.4,2,2,1223,1,299.735235,1,2,4,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,1,7,4,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,6,3,2,4,1,3,2,7,1,5,1,-1,2,...,2,1,2,2,2,4,1,10,3,1,2,-1,1,-1,2,2,1,2,2,2,1,2,2,2,694.0,1725.0,69.4,172.5,2,3,1223,1,355.170081,1,2,4,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,2,5,4,2,-1,2,1,1,1,1,2,5,1,4,1,2,2,6,5,6,3,5,2,6,2,1,5,1,5,1,...,2,1,4,2,2,4,1,3,2,2,2,13,1,-1,1,2,1,1,1,1,1,1,1,1,789.0,1675.0,78.9,167.5,2,4,1223,1,299.735235,1,2,4,1,5,1,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16551,2015,76,5,1,2,4,12,8,8,1,1,7,5,2,-1,1,1,1,1,1,1,3,1,3,1,2,2,6,3,8,7,5,8,7,5,1,2,1,-1,2,...,2,1,1,1,1,4,1,2,1,1,1,7,1,1,2,2,2,2,2,2,1,2,2,2,567.0,1679.0,56.7,167.9,2,2,521,5,447.815539,1,2,4,1,16552,371,117
16552,2015,76,5,1,1,1,15,7,5,1,1,7,4,2,-1,2,1,1,1,1,1,3,1,3,1,1,2,5,4,6,2,4,1,6,1,1,6,4,-1,2,...,2,1,2,1,1,4,1,2,1,2,5,12,1,-1,2,2,1,2,2,2,2,2,2,2,619.0,1858.0,61.9,185.8,2,2,521,5,592.507440,1,2,4,1,16553,371,117
16553,2015,76,5,1,1,4,13,11,7,1,1,7,4,2,-1,1,2,1,1,1,2,3,1,3,1,2,2,5,4,8,1,8,2,8,1,1,5,5,-1,2,...,2,4,1,5,5,4,1,7,3,1,5,-1,1,-1,2,2,2,2,2,2,1,2,2,2,609.0,1594.0,60.9,159.4,2,3,521,5,401.131512,1,2,4,1,16554,371,117
16554,2015,76,5,1,1,4,14,8,6,1,1,7,4,2,-1,1,1,1,1,1,1,3,2,3,1,1,2,5,3,7,5,6,4,8,7,3,1,5,-1,2,...,2,3,2,3,4,4,2,-1,1,1,2,7,1,-1,1,1,2,2,2,2,1,2,2,2,473.0,1578.0,47.3,157.8,2,2,521,5,442.718091,1,2,4,1,16555,371,117


#FUNÇÕES USADAS

Cria uma tabela com coluna de frequencia e porcentagem, podendo também aceitar um dicionário de dados para substituir campos com códigos da pergunta, além de um título.

In [4]:
def formated_table(dataset : pd.DataFrame, dicionario : dict  = None, title_index :str = None):
  frequency = dataset.value_counts()
  percentage = (dataset.value_counts(normalize=True) * 100).round(2)
  final_dataset = pd.DataFrame({"Frequencia":frequency,"Porcentagem":percentage})

  if dicionario:
    final_dataset.rename(index = dicionario, inplace = True)
  
  if title_index:
    final_dataset.rename_axis(title_index,axis=1,inplace=True)
  
  return final_dataset

#ANALISES

**Percentual de alunos do sexo Masculino e Feminino**

Primeiramente, precisamos deixar bem claro o que temos nos nossos dados. A informações abaixo nos mostram a distribuição dos nossos alunos em relação a sexo e idade. Essas são informações básicas que servirão para nos dar um entendimento inicial sobre o que estamos analisando com que público e com que idade.

In [5]:
dicio_vb01001 = {
    1:"MASCULINO",
    2:"FEMININO"
}
DF_sexo_alunos = formated_table(dados["VB01001"],dicio_vb01001,"Sexo")

In [6]:
dicio_vb01003 = {
  11:	"11 anos ou menos",
  12:	"12 anos",
  13:	"13 anos",
  14:	"14 anos",
  15:	"15 anos",
  16:	"16 anos", 
  17:	"17 anos",
  18:	"18 anos",
  19:	"19 anos ou mais"
}

DF_idade_alunos = formated_table(dados["VB01003"],dicio_vb01003,"Idade")

In [37]:
fig = px.bar(DF_idade_alunos,x=DF_idade_alunos.index,y='Porcentagem',title="Porcentagem de idades dos alunos entrevistados",
       labels={"x":"Idades Entrevistadas","Porcentagem":"Porcentagem (%)"})

fig.show()

In [7]:
DF_idade_pelo_sexo = (pd.crosstab(dados['VB01003'],dados['VB01001'],normalize='index') *100).round(2)
DF_idade_pelo_sexo.rename(index=dicio_vb01003,inplace=True)
DF_idade_pelo_sexo = DF_idade_pelo_sexo.rename(columns=dicio_vb01001)

In [36]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type":"bar"},{"type":"pie"}]])


fig.add_bar(x= DF_idade_pelo_sexo.index, 
            y= DF_idade_pelo_sexo['MASCULINO'],
            name = 'IDADE SEXO MASCULINO',
            showlegend=True,
            row=1, col=1)

fig.add_bar(x= DF_idade_pelo_sexo.index,
            y= DF_idade_pelo_sexo['FEMININO'],
            name = 'IDADE SEXO FEMININO',
            showlegend=True,
            row=1, col=1)


fig.add_pie(values= DF_sexo_alunos['Porcentagem'],
              labels = DF_sexo_alunos.index,
              hole=.4,
              textfont=dict(size=20),
              hoverinfo='label+percent',
              textinfo='value',
                 
              row=1, col=2)


fig.update_layout(height=600,
                  title='Disitribuição das Idades e sexo dos alunos entrevistados',
                  barmode= 'stack',showlegend=True)


Apesar do gráfico de pizza não ser uma boa para situações gerais, utilizá-lo para demonstrar a quase igualdade de sexos que temos em nossas amostras. No primeiro gráfico, temos a distribuição dos alunos quanto ao sexo levando em conta suas idades. Passe o mouse por cima para ver melhor os dados.

Vamos buscar entender como andar o comportamento dos jovens nesse período. Como suas atitudes podem estar relacionadas a sua saúde mental.


Primeiro vamos ver a frequência de alunos que faltaram aula nos últimos 30 dias sem autorização dos responsáveis

In [9]:
formated_table(dados["VB07001"])

,Frequencia,Porcentagem
1,12694,76.67
2,2492,15.05
3,882,5.33
4,242,1.46
5,193,1.17
99,53,0.32


NOS ÚLTIMOS 30 DIAS, com que frequência seus pais ou responsáveis sabiam realmente o que você estava fazendo em seu tempo livre?

In [10]:
formated_table(dados["VB07002"])

,Frequencia,Porcentagem
5,6872,41.51
4,4277,25.83
3,2306,13.93
1,1672,10.10
2,1348,8.14
99,81,0.49


Agora, vejamos mais especificamente essa mesma pergunta com base na idade dos alunos. Suponho eu que conforme a idade, a taxa de "Nunca" cresça. vamos ver

In [11]:
(pd.crosstab(dados["VB07002"],dados["VB01003"],normalize="columns") *100).round(2)

VB01003,11,12,13,14,15,16,17,18,19
VB07002,,,,,,,,,
1,8.93,8.20,14.06,13.31,8.82,7.22,7.94,11.55,17.17
2,5.31,6.34,7.89,9.66,9.07,8.59,9.43,11.37,10.80
3,15.78,16.70,12.57,12.47,12.70,12.69,13.60,15.99,15.51
4,20.61,24.56,23.23,25.79,30.56,28.99,30.58,23.27,16.07
5,48.55,43.66,41.35,38.35,38.72,42.20,38.13,37.66,39.89
99,0.82,0.53,0.90,0.42,0.12,0.31,0.33,0.18,0.55


NOS ÚLTIMOS 30 DIAS, com que frequência seus pais ou responsáveis entenderam seus problemas e preocupações?

In [12]:
formated_table(dados["VB07004"])

,Frequencia,Porcentagem
5,5070,30.62
3,3608,21.79
4,2818,17.02
1,2493,15.06
2,2476,14.96
99,91,0.55


NOS ÚLTIMOS 30 DIAS, com que frequência algum dos seus colegas de escola te esculacharam, zoaram, mangaram, intimidaram ou caçoaram tanto que você ficou magoado, incomodado, aborrecido, ofendido ou humilhado?

In [13]:
formated_table(dados['VB07007'])

,Frequencia,Porcentagem
1,9147,55.25
2,3541,21.39
3,2550,15.40
4,670,4.05
5,545,3.29
99,103,0.62


NOS ÚLTIMOS 30 DIAS, qual o motivo/causa de seus colegas terem te esculachado, zombado, zoado, caçoado, mangado, intimidado ou humilhado?

In [14]:
formated_table(dados['VB07008'])

,Frequencia,Porcentagem
-1,9250,55.87
7,4489,27.11
4,1110,6.70
3,705,4.26
1,403,2.43
2,241,1.46
5,160,0.97
6,119,0.72
99,79,0.48


In [15]:
pd.crosstab(dados["VB07008"],dados["VB01001"])
#pessoas do sexo feminino zoam mais pela aparência do corpo enquanto que do sexo
#masculino zoam mais pela raça ou cor

VB01001,1,2
VB07008,,
-1,4543,4707
1,264,139
2,137,104
3,389,316
4,493,617
5,115,45
6,68,51
7,2234,2255
99,44,35


In [16]:
pd.crosstab(dados["VB07008"],dados["VB01003"])

VB01003,11,12,13,14,15,16,17,18,19
VB07008,,,,,,,,,
-1,1142,1329,1401,1147,1390,1362,880,347,252
1,55,62,71,47,64,43,37,17,7
2,28,33,35,34,39,29,22,9,12
3,77,155,133,83,94,85,56,17,5
4,130,211,197,160,147,136,95,20,14
5,19,22,21,16,18,30,19,10,5
6,10,21,16,11,21,19,13,7,1
7,594,785,673,626,638,560,412,136,65
99,17,16,14,9,14,6,3,0,0


Você já sofreu bullying

In [17]:
formated_table(dados['VB07010'])

,Frequencia,Porcentagem
2,8533,51.54
1,7584,45.81
3,387,2.34
99,52,0.31


In [18]:
pd.crosstab(dados['VB07010'],dados['VB01001'],normalize="index").round(2) *100

VB01001,1,2
VB07010,,
1,48.0,52.0
2,51.0,49.0
3,59.0,41.0
99,62.0,38.0


In [19]:
pd.crosstab(dados['VB07010'],dados['VB01003'],normalize="columns").round(2) *100

VB01003,11,12,13,14,15,16,17,18,19
VB07010,,,,,,,,,
1,41.0,48.0,46.0,47.0,46.0,46.0,48.0,45.0,40.0
2,55.0,49.0,51.0,51.0,52.0,52.0,50.0,53.0,56.0
3,4.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,4.0
99,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


NOS ÚLTIMOS 12 MESES com que frequência tem se sentido sozinho(a)? 

In [20]:
formated_table(dados["VB12001"])

,Frequencia,Porcentagem
1,5805,35.06
3,4580,27.66
2,3584,21.65
4,1601,9.67
5,920,5.56
99,66,0.40


In [21]:
pd.crosstab(dados["VB12001"],dados["VB01001"],normalize='columns').round(2) *100

VB01001,1,2
VB12001,,
1,43.0,27.0
2,23.0,21.0
3,23.0,32.0
4,7.0,12.0
5,4.0,7.0
99,1.0,0.0


In [22]:
(pd.crosstab(dados["VB12001"],dados["VB01003"],normalize='columns').round(2) *100)

VB01003,11,12,13,14,15,16,17,18,19
VB12001,,,,,,,,,
1,44.0,40.0,42.0,37.0,29.0,28.0,23.0,29.0,31.0
2,19.0,20.0,19.0,20.0,24.0,24.0,27.0,23.0,20.0
3,24.0,27.0,23.0,27.0,29.0,31.0,33.0,33.0,32.0
4,7.0,7.0,9.0,10.0,12.0,12.0,11.0,10.0,9.0
5,5.0,6.0,6.0,6.0,6.0,5.0,5.0,5.0,8.0
99,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


NOS ÚLTIMOS 12 MESES, com que frequência você não conseguiu dormir à noite porque algo o(a) preocupava muito? 

In [23]:
formated_table(dados["VB12002"])

,Frequencia,Porcentagem
1,5701,34.43
2,4736,28.61
3,4149,25.06
4,1374,8.30
5,519,3.13
99,77,0.47


In [24]:
pd.crosstab(dados["VB12002"],dados["VB01001"],normalize='columns').round(2) *100

VB01001,1,2
VB12002,,
1,42.0,27.0
2,28.0,29.0
3,21.0,29.0
4,6.0,11.0
5,3.0,4.0
99,1.0,0.0


In [25]:
(pd.crosstab(dados["VB12002"],dados["VB01003"],normalize='columns').round(2) *100)

VB01003,11,12,13,14,15,16,17,18,19
VB12002,,,,,,,,,
1,41.0,38.0,41.0,37.0,31.0,28.0,25.0,25.0,26.0
2,23.0,26.0,26.0,27.0,31.0,34.0,34.0,32.0,28.0
3,25.0,24.0,22.0,24.0,26.0,26.0,28.0,29.0,32.0
4,7.0,9.0,7.0,8.0,9.0,9.0,9.0,12.0,9.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0
99,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
